In [1]:
import numpy as np
import pandas as pd
import networkx as nx
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn.metrics import confusion_matrix

In [2]:
#loading the feature matrix and labels 
feature_matrix = np.load('D:/NLP Project/Data/feature_matrix.npy')
class_labels = np.load('D:/NLP Project/Data/class_lables.npy')
indices = np.arange(len(feature_matrix))

In [3]:
graph_df = pd.read_csv('D:/NLP Project/Data/edge_list.csv')
#loading the edgelist and constructing the url graph
g = nx.from_pandas_edgelist(graph_df,'url1','url2',edge_attr='edge_weight')

In [4]:
nodes = list(g.nodes)
len(nodes)

17601

In [5]:
iter_labels = class_labels #making a copy of class labels for updations during iteration
# Contains true labels of all documents.

In [6]:
np.unique(class_labels,return_counts=True)

(array([0, 1, 2, 3]), array([4440, 4236, 5143, 3782], dtype=int64))

In [7]:
nx.info(g)

'Graph with 17601 nodes and 45259074 edges'

### Model 1 : Naive Bayes

In [8]:
testSize = [0.8,0.6,0.4,0.2]
accuracy_list_NB = []
precision_list_NB = []
recall_list_NB = []
conf_mat_NB = []

#loading the feature matrix and labels 
feature_matrix = np.load('D:/NLP Project/Data/feature_matrix.npy')
class_labels = np.load('D:/NLP Project/Data/class_lables.npy')
indices = np.arange(len(feature_matrix))

for k in testSize:
    print('For test size: ',k)
    X_train, X_test, y_train, y_test = train_test_split(feature_matrix, class_labels,test_size=k, random_state=0, stratify=class_labels)
    clf = GaussianNB() # Bootstrapping using Naive Bayes as Base Classifier
    clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    confusion_mat = confusion_matrix(y_test,y_pred)

    conf_mat_NB.append(confusion_mat)
    accuracy_list_NB.append(metrics.accuracy_score(y_test,y_pred))
    precision_list_NB.append(metrics.precision_score(y_test,y_pred,average='macro',zero_division=0))
    recall_list_NB.append(metrics.recall_score(y_test,y_pred,average='macro'))

    # Micro
    precision_list_NB.append(metrics.precision_score(y_test,y_pred,average='micro',zero_division=0))
    recall_list_NB.append(metrics.recall_score(y_test,y_pred,average='micro'))


For test size:  0.8
For test size:  0.6
For test size:  0.4
For test size:  0.2


In [9]:
accuracy_list_NB = ["%.2f" % elem for elem in accuracy_list_NB]
precision_list_NB = ["%.2f" % elem for elem in precision_list_NB]
recall_list_NB = ["%.2f" % elem for elem in recall_list_NB]

print(accuracy_list_NB)
print(precision_list_NB)
print(recall_list_NB)
print(conf_mat_NB)

['0.64', '0.64', '0.64', '0.66']
['0.65', '0.64', '0.65', '0.64', '0.66', '0.64', '0.69', '0.66']
['0.65', '0.64', '0.65', '0.64', '0.65', '0.64', '0.67', '0.66']
[array([[2327,  425,  653,  147],
       [ 510, 2068,  476,  335],
       [ 899,  746, 2338,  131],
       [  79,  511,  123, 2313]], dtype=int64), array([[1994,  178,  297,  195],
       [ 540, 1300,  252,  450],
       [1027,  296, 1556,  207],
       [  65,  235,   66, 1903]], dtype=int64), array([[1421,   84,  126,  145],
       [ 372,  775,  123,  425],
       [ 737,  137,  980,  203],
       [  35,  131,   26, 1321]], dtype=int64), array([[732,  34,  55,  67],
       [171, 379,  60, 237],
       [336,  60, 516, 117],
       [  7,  52,   5, 693]], dtype=int64)]


### Model 1 : ICA - NB with Label Counts

In [10]:
##Bootstrapping
testSize = [0.8,0.6,0.4,0.2]
accuracy_list_ICA_Labels = []
precision_list_ICA_Labels = []
recall_list_ICA_Labels = []
conf_mat_ICA_Labels = []

feature_matrix = np.load('D:/NLP Project/Data/feature_matrix.npy')
class_labels = np.load('D:/NLP Project/Data/class_lables.npy')
indices = np.arange(len(feature_matrix))

for k in testSize:
    print('For test size: ',k)
    X_train, X_test, y_train, y_test,idx_train,idx_test = train_test_split(feature_matrix, class_labels, indices,test_size=k, random_state=0, stratify=class_labels)
    clf = GaussianNB() # Bootstrapping using Naive Bayes as Base Classifier
    clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test) # y-pred is bootstrapped labels
    iter_labels = class_labels
    np.put(iter_labels,idx_test,y_pred)  # updating labels of test data with the predecited labels 
    iter_labels_list = list(iter_labels)
    distinct_labels = sorted(list(set(iter_labels_list)))

    # ICA starting

    adj_feats_train = np.zeros((len(X_train),len(distinct_labels)))
    adj_feats_test  = np.zeros((len(X_test),len(distinct_labels)))

    ########################################################################
    ##constructing additional features, train and Iterate until stabilized##

    #updating the adjacent features for training nodes
    for i in range(len(idx_train)):
        adjs = list(g[idx_train[i]])
        adjs = [int(nei) for nei in adjs]
        labels_of_adjacent_nodes = [iter_labels_list[nei] for nei in adjs]

        for j in range(len(distinct_labels)):
            this_label = distinct_labels[j]
            cnt_of_adjacent_labels = labels_of_adjacent_nodes.count(this_label) ## Voting by neighbor nodes.
            adj_feats_train[i][this_label] = cnt_of_adjacent_labels
        
          
    X_train_updated = np.concatenate((X_train,adj_feats_train),axis = 1)

    #updating the adjancent features for test nodes
    for i in range(len(idx_test)):
        adjs = list(g[idx_test[i]])
        adjs = [int(nei) for nei in adjs]
        labels_of_adjacent_nodes = [iter_labels_list[nei] for nei in adjs]

        for j in range(len(distinct_labels)):
            this_label = distinct_labels[j]
            cnt_of_adjacent_labels = labels_of_adjacent_nodes.count(this_label) ## Voting by neighbor nodes.
            adj_feats_test[i][this_label] = cnt_of_adjacent_labels

    X_test_updated = np.concatenate((X_test,adj_feats_test),axis = 1)  

    #learning the new model on updated feature matrix with adjacent labels
    clf_updated = GaussianNB()
    clf_updated.fit(X_train_updated,y_train)
    #print("\nStarting ICA Loop: ...\n")
    #staring the ICA inference loop
    loop_var = 0
    iter_var = 0
    y_pred_current = y_pred
    while (loop_var == 0 and iter_var < 15):
            y_pred_updated = clf_updated.predict(X_test_updated)
            if(np.array_equal(y_pred_current, y_pred_updated)):
                #algorithm stabilized
                #print("ICA Stabilized")
                loop_var = 1        
            else:
                loop_var = 0
                iter_var += 1
                #print("ICA Loop: "+str(iter_var))
                
                #updating the labels for test nodes with new predictions
                for i in range(len(idx_test)):
                    iter_labels_list[idx_test[i]] = y_pred_updated[i]
                
                #updating the adjacent features for test nodes
                for i in range(len(idx_test)):
                    adjs = list(g[idx_test[i]])
                    adjs = [int(nei) for nei in adjs]
                    labels_of_adjacent_nodes = [iter_labels_list[nei] for nei in adjs]

                    for j in range(len(distinct_labels)):
                        this_label = distinct_labels[j]
                        cnt_of_adjacent_labels = labels_of_adjacent_nodes.count(this_label) ## Voting by neighbor nodes.
                        adj_feats_test[i][this_label] = cnt_of_adjacent_labels
                                        
                X_test_updated = np.concatenate((X_test,adj_feats_test),axis = 1)
                y_pred_current = y_pred_updated   
    
    #print('No. of iterations ICA ran: ',iter_var)            
    final_predictions = y_pred_updated

    #print('\nMetrics for Iterative Classification Algorithm for train size {:.1f}:\n'.format(1-k))
    # ICA
    # Macro
    confusion_mat = confusion_matrix(y_test,y_pred_updated)
    conf_mat_ICA_Labels.append(confusion_mat)

    accuracy_list_ICA_Labels.append(metrics.accuracy_score(y_test,y_pred_updated))
    precision_list_ICA_Labels.append(metrics.precision_score(y_test,y_pred_updated,average='macro',zero_division=0))
    recall_list_ICA_Labels.append(metrics.recall_score(y_test,y_pred_updated,average='macro'))

    # Micro
    precision_list_ICA_Labels.append(metrics.precision_score(y_test,y_pred_updated,average='micro',zero_division=0))
    recall_list_ICA_Labels.append(metrics.recall_score(y_test,y_pred_updated,average='micro'))


For test size:  0.8
For test size:  0.6
For test size:  0.4
For test size:  0.2


In [11]:
accuracy_list_ICA_Labels = ["%.2f" % elem for elem in accuracy_list_ICA_Labels]
precision_list_ICA_Labels = ["%.2f" % elem for elem in precision_list_ICA_Labels]
recall_list_ICA_Labels = ["%.2f" % elem for elem in recall_list_ICA_Labels]

print(accuracy_list_ICA_Labels)
print(precision_list_ICA_Labels)
print(recall_list_ICA_Labels)
print(conf_mat_ICA_Labels)

['0.72', '0.78', '0.80', '0.81']
['0.73', '0.72', '0.79', '0.78', '0.80', '0.80', '0.81', '0.81']
['0.73', '0.72', '0.79', '0.78', '0.79', '0.80', '0.80', '0.81']
[array([[2654,  286,  522,   90],
       [ 425, 2226,  446,  292],
       [ 729,  536, 2744,  105],
       [  55,  327,   63, 2581]], dtype=int64), array([[2441,  157,  172,   52],
       [ 286, 1940,  225,  307],
       [ 513,  248, 1890,  121],
       [  92,  107,   12, 1998]], dtype=int64), array([[1820,  118,   97,   52],
       [ 170, 1188,  112,  196],
       [ 321,  119, 1164,   58],
       [  78,   82,   18, 1448]], dtype=int64), array([[984,  69,  47,  18],
       [ 80, 548,  36, 111],
       [134,  46, 535,  39],
       [ 45,  40,   4, 785]], dtype=int64)]


### Model 2 : ICA - NB with Sum of Edge Weights 

In [12]:
testSize = [0.8,0.6,0.4,0.2]
accuracy_list_ICA_EW= []
precision_list_ICA_EW = []
recall_list_ICA_EW = []
conf_mat_ICA_EW = []

feature_matrix = np.load('D:/NLP Project/Data/feature_matrix.npy')
class_labels = np.load('D:/NLP Project/Data/class_lables.npy')
indices = np.arange(len(feature_matrix))

for k in testSize:
    print('For test size: ',k)
    X_train, X_test, y_train, y_test,idx_train,idx_test = train_test_split(feature_matrix, class_labels, indices,test_size=k, random_state=0, stratify=class_labels)
    clf = GaussianNB()
    clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    iter_labels = class_labels
    np.put(iter_labels,idx_test,y_pred)  # updating labels of test data with the predecited labels 
    iter_labels_list = list(iter_labels)
    distinct_labels = sorted(list(set(iter_labels_list)))

    # ICA starting

    adj_feats_train = np.zeros((len(X_train),len(distinct_labels)))
    adj_feats_test  = np.zeros((len(X_test),len(distinct_labels)))

    ########################################################################
    ##constructing additional features, train and Iterate until stabilized##

    #updating the adjancent features for training nodes
    for i in range(len(idx_train)):
        adjs = list(g[idx_train[i]])
        adjs = [int(nei) for nei in adjs]
        edge_weight_adj_nodes = [0]*len(distinct_labels)

        for j in range(len(adjs)):
            this_label = iter_labels_list[adjs[j]]
            edge_weight_adj_nodes[this_label] +=  int(g[idx_train[i]][adjs[j]]['edge_weight'])/len(g[idx_train[i]])
            adj_feats_train[i][this_label] = edge_weight_adj_nodes[this_label]
        
          
    X_train_updated = np.concatenate((X_train,adj_feats_train),axis = 1)

    #updating the adjancent features for test nodes
    for i in range(len(idx_test)):
        adjs = list(g[idx_test[i]])
        adjs = [int(nei) for nei in adjs]
        edge_weight_adj_nodes = [0]*len(distinct_labels)

        for j in range(len(adjs)):
            this_label = iter_labels_list[adjs[j]]
            edge_weight_adj_nodes[this_label] +=  int(g[idx_test[i]][adjs[j]]['edge_weight'])/len(g[idx_test[i]])
            adj_feats_test[i][this_label] = edge_weight_adj_nodes[this_label]

    X_test_updated = np.concatenate((X_test,adj_feats_test),axis = 1)  

    #learning the new model on updated feature matrix with adjacent labels
    clf_updated = GaussianNB()
    clf_updated.fit(X_train_updated,y_train)
    #print("\nStarting ICA Loop: ...\n")
    #staring the ICA inference loop
    loop_var = 0
    iter_var = 0
    y_pred_current = y_pred
    while (loop_var == 0 and iter_var < 15):
            y_pred_updated = clf_updated.predict(X_test_updated)
            if(np.array_equal(y_pred_current, y_pred_updated)):
                #algorithm stabilized
                #print("ICA Stabilized")
                loop_var = 1        
            else:
                loop_var = 0
                iter_var += 1
                #print("ICA Loop: "+str(iter_var))
                
                #updating the labels for test nodes with new predictions
                for i in range(len(idx_test)):
                    iter_labels_list[idx_test[i]] = y_pred_updated[i]
                
                #updating the adjancent features for test nodes
                for i in range(len(idx_test)):
                    adjs = list(g[idx_test[i]])
                    adjs = [int(nei) for nei in adjs]
                    edge_weight_adj_nodes = [0]*len(distinct_labels)
    
                    for j in range(len(adjs)):
                        this_label = iter_labels_list[adjs[j]]
                        edge_weight_adj_nodes[this_label] +=  int(g[idx_test[i]][adjs[j]]['edge_weight'])/len(g[idx_test[i]])
                        adj_feats_test[i][this_label] = edge_weight_adj_nodes[this_label]     
                    
                X_test_updated = np.concatenate((X_test,adj_feats_test),axis = 1)
                y_pred_current = y_pred_updated   
    #print('No. of iterations ICA ran: ',iter_var)            
    final_predictions = y_pred_updated

    #print('ICA - Edge Weight Confusion matrix:\n')
    confusion_mat = confusion_matrix(y_test,y_pred_updated)
    conf_mat_ICA_EW.append(confusion_mat)
    #print(confusion_mat)

    #print('\nMetrics for Iterative Classification Algorithm for train size {:.1f}:\n'.format(1-k))
    # ICA
    # Macro
    accuracy_list_ICA_EW.append(metrics.accuracy_score(y_test,y_pred_updated))
    precision_list_ICA_EW.append(metrics.precision_score(y_test,y_pred_updated,average='macro',zero_division=0))
    recall_list_ICA_EW.append(metrics.recall_score(y_test,y_pred_updated,average='macro'))

    # Micro
    precision_list_ICA_EW.append(metrics.precision_score(y_test,y_pred_updated,average='micro',zero_division=0))
    recall_list_ICA_EW.append(metrics.recall_score(y_test,y_pred_updated,average='micro'))


For test size:  0.8
For test size:  0.6
For test size:  0.4
For test size:  0.2


In [13]:
accuracy_list_ICA_EW = ["%.2f" % elem for elem in accuracy_list_ICA_EW]
precision_list_ICA_EW = ["%.2f" % elem for elem in precision_list_ICA_EW]
recall_list_ICA_EW = ["%.2f" % elem for elem in recall_list_ICA_EW]

print(accuracy_list_ICA_EW)
print(precision_list_ICA_EW)
print(recall_list_ICA_EW)
print(conf_mat_ICA_EW)

['0.64', '0.75', '0.81', '0.86']
['0.65', '0.64', '0.75', '0.75', '0.82', '0.81', '0.88', '0.86']
['0.65', '0.64', '0.76', '0.75', '0.81', '0.81', '0.85', '0.86']
[array([[2327,  425,  653,  147],
       [ 510, 2068,  476,  335],
       [ 899,  746, 2338,  131],
       [  79,  511,  123, 2313]], dtype=int64), array([[2341,  149,  206,  126],
       [ 375, 1782,  219,  382],
       [ 566,  232, 1842,  132],
       [  55,  162,   40, 1952]], dtype=int64), array([[1905,   44,   56,   82],
       [ 190, 1156,   83,  237],
       [ 333,  144, 1124,   61],
       [  27,   33,   14, 1552]], dtype=int64), array([[1071,    6,    7,   34],
       [  61,  568,   26,  120],
       [ 119,   52,  548,   35],
       [   7,   11,    3,  853]], dtype=int64)]


### Model 3 : ICA-NB with Combined Features

In [14]:
testSize = [0.8,0.6,0.4,0.2]
accuracy_list_ICA_Combined = []
precision_list_ICA_Combined = []
recall_list_ICA_Combined = []
conf_matrix_ICA_Combined = []

feature_matrix = np.load('D:/NLP Project/Data/feature_matrix.npy')
class_labels = np.load('D:/NLP Project/Data/class_lables.npy')
indices = np.arange(len(feature_matrix))

for k in testSize:
    print('For test size: ',k)
    X_train, X_test, y_train, y_test,idx_train,idx_test = train_test_split(feature_matrix, class_labels, indices,test_size=k, random_state=0, stratify=class_labels)
    clf = GaussianNB()
    clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    iter_labels = class_labels
    np.put(iter_labels,idx_test,y_pred)  # updating labels of test data with the predecited labels 
    iter_labels_list = list(iter_labels)
    distinct_labels = sorted(list(set(iter_labels_list)))

    # ICA starting

    adj_feats_train = np.zeros((len(X_train),2*len(distinct_labels))) # Multiply by 2, since we have 8 additional features combined.
    adj_feats_test  = np.zeros((len(X_test),2*len(distinct_labels)))

    ########################################################################
    ##constructing additional features, train and Iterate until stabilized##

    #updating the adjancent features for training nodes
    for i in range(len(idx_train)):       
        adjs = list(g[idx_train[i]])
        adjs = [int(nei) for nei in adjs]
        edge_weight_adj_nodes = [0]*len(distinct_labels)
        labels_of_adjacent_nodes = [iter_labels_list[nei] for nei in adjs]
        
        for j in range(len(adjs)):
            this_label = iter_labels_list[adjs[j]]
            edge_weight_adj_nodes[this_label] +=  int(g[idx_train[i]][adjs[j]]['edge_weight'])/len(g[idx_train[i]])
            adj_feats_train[i][this_label] = edge_weight_adj_nodes[this_label]
        
        for k in range(len(distinct_labels)):
            this_label = distinct_labels[k]
            cnt_of_adjacent_labels = labels_of_adjacent_nodes.count(this_label) ## Voting by neighbor nodes.
            adj_feats_train[i][this_label+len(distinct_labels)] = cnt_of_adjacent_labels
          
    X_train_updated = np.concatenate((X_train,adj_feats_train),axis = 1)

    #updating the adjancent features for test nodes
    for i in range(len(idx_test)):       
        adjs = list(g[idx_test[i]])
        adjs = [int(nei) for nei in adjs]
        edge_weight_adj_nodes = [0]*len(distinct_labels)
        labels_of_adjacent_nodes = [iter_labels_list[nei] for nei in adjs]
        
        for j in range(len(adjs)):
            this_label = iter_labels_list[adjs[j]]
            edge_weight_adj_nodes[this_label] +=  int(g[idx_test[i]][adjs[j]]['edge_weight'])/len(g[idx_test[i]])
            adj_feats_test[i][this_label] = edge_weight_adj_nodes[this_label]
        
        for k in range(len(distinct_labels)):
            this_label = distinct_labels[k]
            cnt_of_adjacent_labels = labels_of_adjacent_nodes.count(this_label) ## Voting by neighbor nodes.
            adj_feats_test[i][this_label+len(distinct_labels)] = cnt_of_adjacent_labels

    X_test_updated = np.concatenate((X_test,adj_feats_test),axis = 1)  

    #learning the new model on updated feature matrix with adjacent labels
    clf_updated = GaussianNB()
    clf_updated.fit(X_train_updated,y_train)
    #print("\nStarting ICA Loop: ...\n")
    #staring the ICA inference loop
    loop_var = 0
    iter_var = 0
    y_pred_current = y_pred
    while (loop_var == 0 and iter_var < 15):
            y_pred_updated = clf_updated.predict(X_test_updated)
            if(np.array_equal(y_pred_current, y_pred_updated)):
                #algorithm stabilized
                #print("ICA Stabilized")
                loop_var = 1        
            else:
                loop_var = 0
                iter_var += 1
                #print("ICA Loop: "+str(iter_var))
                
                #updating the labels for test nodes with new predictions
                for i in range(len(idx_test)):
                    iter_labels_list[idx_test[i]] = y_pred_updated[i]
                
                #updating the adjancent features for test nodes
                for i in range(len(idx_test)):       
                    adjs = list(g[idx_test[i]])
                    adjs = [int(nei) for nei in adjs]
                    edge_weight_adj_nodes = [0]*len(distinct_labels)
                    labels_of_adjacent_nodes = [iter_labels_list[nei] for nei in adjs]
                    
                    for j in range(len(adjs)):
                        this_label = iter_labels_list[adjs[j]]
                        edge_weight_adj_nodes[this_label] +=  int(g[idx_test[i]][adjs[j]]['edge_weight'])/len(g[idx_test[i]])
                        adj_feats_test[i][this_label] = edge_weight_adj_nodes[this_label]
                    
                    for k in range(len(distinct_labels)):
                        this_label = distinct_labels[k]
                        cnt_of_adjacent_labels = labels_of_adjacent_nodes.count(this_label) ## Voting by neighbor nodes.
                        adj_feats_test[i][this_label+len(distinct_labels)] = cnt_of_adjacent_labels       
                                
                X_test_updated = np.concatenate((X_test,adj_feats_test),axis = 1)
                y_pred_current = y_pred_updated   
    print('No. of iterations ICA ran: ',iter_var)            
    final_predictions = y_pred_updated

    #print('ICA - Edge Weight Confusion matrix:\n')
    confusion_mat = confusion_matrix(y_test,y_pred_updated)
    conf_matrix_ICA_Combined.append(confusion_mat)
    #print(confusion_mat)

    #print('\nMetrics for Iterative Classification Algorithm for train size {:.1f}:\n'.format(1-k))
    # ICA
    # Macro
    accuracy_list_ICA_Combined.append(metrics.accuracy_score(y_test,y_pred_updated))
    precision_list_ICA_Combined.append(metrics.precision_score(y_test,y_pred_updated,average='macro',zero_division=0))
    recall_list_ICA_Combined.append(metrics.recall_score(y_test,y_pred_updated,average='macro'))

    # Micro
    precision_list_ICA_Combined.append(metrics.precision_score(y_test,y_pred_updated,average='micro',zero_division=0))
    recall_list_ICA_Combined.append(metrics.recall_score(y_test,y_pred_updated,average='micro'))


For test size:  0.8
No. of iterations ICA ran:  2
For test size:  0.6
No. of iterations ICA ran:  2
For test size:  0.4
No. of iterations ICA ran:  2
For test size:  0.2
No. of iterations ICA ran:  1


In [15]:
accuracy_list_ICA_Combined = ["%.2f" % elem for elem in accuracy_list_ICA_Combined]
precision_list_ICA_Combined = ["%.2f" % elem for elem in precision_list_ICA_Combined]
recall_list_ICA_Combined = ["%.2f" % elem for elem in recall_list_ICA_Combined]

print(accuracy_list_ICA_Combined)
print(precision_list_ICA_Combined)
print(recall_list_ICA_Combined)
print(conf_matrix_ICA_Combined)

['0.73', '0.78', '0.80', '0.81']
['0.73', '0.73', '0.79', '0.78', '0.80', '0.80', '0.81', '0.81']
['0.73', '0.73', '0.79', '0.78', '0.79', '0.80', '0.80', '0.81']
[array([[2655,  286,  522,   89],
       [ 424, 2227,  446,  292],
       [ 727,  535, 2747,  105],
       [  55,  327,   63, 2581]], dtype=int64), array([[2441,  157,  172,   52],
       [ 286, 1941,  224,  307],
       [ 513,  247, 1891,  121],
       [  92,  107,   12, 1998]], dtype=int64), array([[1821,  118,   97,   51],
       [ 171, 1188,  111,  196],
       [ 321,  119, 1164,   58],
       [  78,   81,   18, 1449]], dtype=int64), array([[985,  68,  47,  18],
       [ 81, 547,  36, 111],
       [137,  46, 532,  39],
       [ 45,  40,   4, 785]], dtype=int64)]
